In [ ]:
import joblib
import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt

from skopt import BayesSearchCV
from skopt.sampler import Grid

    
df = pd.read_csv('data/data.csv')
df = df.sort_values('query_id')

# EDA

### Посмотрим на корреляционную матрицу

In [ ]:
# Вычисляем корреляционную матрицу
corr_matrix = df.corr()

# Создаем тепловую карту корреляций
plt.figure(figsize=(19, 12))
sns.heatmap(corr_matrix, cmap='coolwarm', fmt=".2f")
plt.title('Карта корреляций между признаками')
plt.show()

### Посмотрим на распределение признаков

In [ ]:
plt.figure()
ax = df.hist(bins=30, edgecolor='black', color='blue', alpha=0.7, figsize=(19, 12))
for axis in ax.flatten():
    axis.axis('off')
    axis.set_title(axis.get_title(), fontsize=8)
    
plt.show()

# Обучение

Из анализа видно, что имеются созависимые признаки и нет нормального распределения, поэтому был выбран Градиентный бустинг (XGBoost)

### Разбиение данных

In [ ]:
X = df.drop(columns=['rank', 'query_id'])
y = df['rank'].values
qid = df['query_id'].values

### Тюнинг гиперпараметров

In [ ]:
params = {
    'objective':"rank:ndcg", 
    'lambdarank_num_pair_per_sample': 5,
    'lambdarank_pair_method': 'topk'
}

opt = BayesSearchCV(
    estimator=xgb.XGBRanker(**params), 
    search_spaces={
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'max_depth': (3, 10),
        'n_estimators': (50, 200),
        'gamma': (0.0, 1.0, 'uniform'),
        'subsample': (0.5, 1.0, 'uniform'),
        'colsample_bytree': (0.5, 1.0, 'uniform'),
        'reg_alpha': (0.0, 1.0, 'uniform'),
        'reg_lambda': (0.0, 1.0, 'uniform'),
        'min_child_weight': (1, 10),
    }, 
    optimizer_kwargs={
        "base_estimator": "GP",
        "initial_point_generator": Grid(
            border="exclude", use_full_layout=False
        ),
    },
    cv=5, 
    n_jobs=-1,
    refit=True
)

opt.fit(X, y, qid=qid)

In [ ]:
print("Best parameters found:", opt.best_params_)
print("Best score found:", opt.best_score_)

### Сохранение лучших параметров и модели

In [ ]:
params.update(dict(opt.best_params_))
joblib.dump(params, 'xgbRanker_hyperparameters.pkl')
joblib.dump(opt.best_estimator_, 'xgbRanker.pkl')

### Загрузка модели

In [ ]:
model = joblib.load('xgbRanker.pkl')

### Обучение модели по лучшим гиперпараметрам

In [ ]:
params = joblib.load('xgbRanker_hyperparameters.pkl')
model = xgb.XGBRanker(**params)
model.fit(X, y, qid=qid)

# Инференс

### Получение оценок модели

In [ ]:
scores = model.predict(X)

### Нормализация оценок

In [ ]:
predictions = (scores - scores.min()) / (scores.max() - scores.min())

### Получение рангов

In [ ]:
predictions = np.round(predictions * y.max()).astype(int)

### Подсчет NDCG@5

In [ ]:
ndcg = df.groupby('query_id').apply(lambda row: model.score(row.drop(columns=['rank', 'query_id']), row['rank'].values))
np.mean(ndcg)